$$
\newcommand{\R}{\mathbb{R}}
\newcommand{\opt}{^{\star}}
$$

# Tutorials

In [1]:
# Import packages
import sys; sys.path.insert(0, "../../")
import numpy as np
import dualbounds as db
from dualbounds.generic import DualBounds

## Minimalist review of dual bounds

In this section, we give a minimal review the dual bounds framework; a more complete review can be found in the first four pages of [Ji et al. (2023)](https://arxiv.org/abs/2310.08115). Users familiar with this content should skip this section. 


Given i.i.d. outcomes $Y_i \in \mathcal{Y}$, treatments $W_i \in \{0,1\}$ and pre-treatment covariates $X_i \in \mathcal{Y}$, dual bounds allow analysts to use machine learning to perform inference on partially identified estimands of the form

$$\theta = E[f(Y_i(0), Y_i(1), X_i)], $$

where $Y_i(1), Y_i(0) \in \mathcal{Y}$ denote potential outcomes. Such estimands are *partially identified* because we never observe the joint law of the potential outcomes, but the data still contains information on the law of $(Y(0), X)$ and $(Y(1), X)$ allowing us to *bound* $\theta$. (More generally, dual bounds can provde bounds on the solutions to generic optimization problems, but we defer discussion of this until later in the tutorial for simplicity).

Thus, to bound $\theta$, one usually must estimate the laws of $(Y(1), X)$ and $(Y(0), X)$, typically using sophisticated machine learning techniques. However, it is not clear if the resulting bounds on $\theta$ will be valid if the learned machine learning models are misspecified or inaccurate. For example, if one models $Y$ as having a linear relationship with $(X,W)$, but in truth $Y$ has a highly nonlinear relationship with $(X,W$), naive approaches could lead to inaccurate inference.

The Dual Bounds methodology is designed to leverage the benefits of sophisticated ML models without sacrificing validity. It has three key properties:
1. Dual bounds can wrap on top of any ML model (e.g. generalized linear models, random forests, neural networks, etc).
2. In randomized experiments, dual bounds yield provably valid confidence intervals on $\theta$, even if the underlying ML model is arbitrarily misspecified or inaccurate. In observational studies, they have remarkably strong double robustness properties (see Section 3.4 of [Ji et al. (2023)](https://arxiv.org/abs/2310.08115)).
3. On the other hand, if the underlying ML model is highly accurate, dual bounds yield extremely sharp confidence bounds in a formal sense (see Section 3.2 of [Ji et al. (2023)](https://arxiv.org/abs/2310.08115)).

We refer the reader to the original paper for more details on how dual bounds work. In the next section, we show how to use dual bounds to perform inference on a variety of estimands 

## The ``DualBounds`` class

The ``DualBounds`` class is the main class in the package. The main usage is as follows. 

**Step 1**: one should initialize the ``DualBounds`` class, which takes as an input (i) the data, (ii) the definition of the function $f$ (which defines the estimand $\theta$), and (iii) a description of the outcome model to use as an input. The user can also input a vector of propensity scores if they are known; else they will be estimated from the data. 

For example, below, we show how to compute $E[\mathbb{I}(Y(1) > Y(0)]$, the probability that the treatment effect is positive.

In [2]:
# Generate synthetic data from a heavy-tailed linear model
data = db.gen_data.gen_regression_data(
    n=900, # Num. datapoints
    p=30, # Num. covariates
    r2=0.95, # population R^2
    tau=3, # average treatment effect
    interactions=True, # ensures treatment effect is heterogenous
    eps_dist='laplace', # heavy-tailed residuals
    sample_seed=123, # random seed
)

# Initialize dual bounds object
dbnd = DualBounds(
    f=lambda y0, y1, x: y0 < y1, # defines the estimand
    X=data['X'], # n x p covariate matrix
    W=data['W'], # n-length treatment vector
    y=data['y'], # n-length outcome vector
    pis=data['pis'], # n-length propensity scores (optional)
    Y_model='ridge', # description of model for Y | X, W
)


**Step 2**: after initialization, the ``compute_dual_bounds`` method fits the underlying outcome model and produces the final estimates and confidence bounds for the sharp partial identification bounds $\theta_L \le \theta \le \theta_U$. 

In [3]:
# Compute dual bounds and observe output
dbnd.compute_dual_bounds(
    nfolds=5, # number of cross-fitting folds
    alpha=0.05 # nominal level
)

100%|████████████████████████████████████████████████████████| 900/900 [00:01<00:00, 828.69it/s]


{'estimates': array([0.57745911, 0.92498299]),
 'ses': array([0.02331765, 0.0129798 ]),
 'cis': array([0.53175736, 0.95042292])}

Note that there are two estimates---a lower and an upper estimate---because $\theta$ is not identified.

Another example below bounds a different estimand, the positive treatment effect $E[\max(Y(1) - Y(0), 0)]$, using a different underlying ML model (a k-nearest neighbors regressor).

In [4]:
dbnd = DualBounds(
    f=lambda y0, y1, x: np.maximum(y1-y0,0), # new estimand
    X=data['X'], # n x p covariate matrix
    W=data['W'], # n-length treatment vector
    y=data['y'], # n-length outcome vector
    pis=data['pis'], # n-length propensity scores (optional)
    Y_model='knn', # description of model for Y | X, W
)
dbnd.compute_dual_bounds(
    alpha=0.1 # nominal level
)

100%|████████████████████████████████████████████████████████| 900/900 [00:01<00:00, 579.67it/s]


{'estimates': array([2.52594772, 4.27917408]),
 'ses': array([0.18387067, 0.15829605]),
 'cis': array([2.22350738, 4.5395479 ])}

## Choosing the outcome model

Dual bounds wrap on top of an underlying model which estimates the conditional distributions of $Y(1) \mid X$ and $Y(0) \mid X$. To allow for maximum flexibility, ``dualbounds`` gives the user four ways to specify the underlying model, listed below in order of increasing flexibility.

### Method 1: String identifiers

The easiest way to choose the model is to use one of the string identifiers, such as ``'ridge', 'lasso', 'elasticnet', 'randomforest', 'knn'`` (see the API reference for a complete list), as demonstrated below.

In [5]:
dbnd = DualBounds(
    Y_model='randomforest', # use random forest model
    f=lambda y0, y1, x: np.maximum(y1-y0,0), # estimand
    X=data['X'], W=data['W'], y=data['y'], # data
)

For binary data, these string identifiers assume a nonparametric model where $Y_i \sim \text{Bern}(\mu(X_i, W_i))$ and the conditional mean function $\mu$ is estimated via one of the models listed above (e.g., a random forest classifier).

For nonbinary data, these string identifiers use a semiparametric regression model:

$$Y_i = \mu(X_i, W_i) + \epsilon_i  $$

where the conditional mean function $\mu(\cdot, \cdot)$ is approximated using one of the models listed above (e.g., a random forest or k-nearest neighbors regressor). All methods automatically create interaction terms between the covariates and the treatment.
 

By default, these string identifiers assume Gaussian homoskedastic residuals, i.e., $\epsilon_i \stackrel{\mathrm{i.i.d.}}{\sim} \mathcal{N}(0, \sigma^2)$. This lightweight approach may not be realistic, but nonetheless we have found that it yields reasonably tight confidence intervals even under misspecification. Of course, it provably yields valid confidence intervals even under arbitrary misspecification of the outcome model.

### Method 2: Input a class inheriting from ``dist_reg.DistReg``

By default, the string identifiers assume Gaussianity and homoskedasticity. Analysts can change these assumptions by initializing either a ``CtsDistReg`` or ``BinaryDistReg`` class and passing it into the ``dualbounds`` class instantiation. 

For example, below we initialize a class which assumes heavy-tailed Laplace errors and heteroskedasticity.

In [6]:
Y_model = db.dist_reg.CtsDistReg(
    model_type='elasticnet', 
    eps_dist='laplace',
    how_transform='interactions', # create interactions btwn X and W
    heterosked=True,
)
dbnd = DualBounds(
    Y_model=Y_model, # use random forest model
    f=lambda y0, y1, x: np.maximum(y1-y0,0), # estimand
    X=data['X'], W=data['W'], y=data['y'], pis=data['pis'], # data
)
dbnd.compute_dual_bounds(alpha=0.05)

100%|████████████████████████████████████████████████████████| 900/900 [00:01<00:00, 611.70it/s]


{'estimates': array([2.75821933, 3.3190261 ]),
 'ses': array([0.23200022, 0.16681487]),
 'cis': array([2.30350725, 3.64597724])}

One can also directly input ``sklearn`` classes. For example, below we show how to use the ``AdaBoostClassifier`` from sklearn for binary data.

In [7]:
import sklearn.ensemble as ensemble
Y_model = db.dist_reg.BinaryDistReg(
    model_type=ensemble.AdaBoostClassifier,
)
dbnd = DualBounds(
    Y_model=Y_model, # use random forest model
    f=lambda y0, y1, x: y0 < y1, # estimand
    y=data['y'] > 0, # make the outcome binary
    X=data['X'], W=data['W'], pis=data['pis'], # data
)
dbnd.compute_dual_bounds(alpha=0.05)

100%|███████████████████████████████████████████████████████| 900/900 [00:00<00:00, 2645.23it/s]


{'estimates': array([0.24130477, 0.37151485]),
 'ses': array([0.02993697, 0.02251905]),
 'cis': array([0.18262938, 0.41565138])}

Analysts can also create custom classes inheritting from ``dualbounds.dist_reg.DistReg``, allowing analysts to use (e.g.) custom conditional variance estimators---see the API reference for more details.

### Method 3: Directly input estimated conditional distributions

For maximum flexibility, one can also directly input predicted conditional distributions of $Y(1) \mid X$ and $Y(0) \mid X$, in the form of a list of batched scipy distributions whose shapes sum to the number of datapoints.

This is illustrated below, although for simplicity the inputs have nothing to do with the true distributions of $Y(1) \mid X$ and $Y(0) \mid X$. Note that in real applications, it is extremely important that the estimates of $Y(1) \mid X$ and $Y(0) \mid X$ must be computed using cross-fitting, otherwise the dual bounds may not be valid.

In [8]:
from scipy import stats
n = len(data['y']) # number of data-points

# Initialize object
dbnd = DualBounds(
    Y_model='lasso', # this will be ignored
    f=lambda y0, y1, x : y0 < y1, # estimand
    y=data['y'], X=data['X'], W=data['W'], pis=data['pis'], # data
)

# Either of the following input formats work
y0_dists = stats.norm(loc=np.zeros(n))
y1_dists = [
    stats.norm(loc=np.zeros(int(n/2)), scale=2), 
    stats.norm(loc=np.zeros(int(n/2)), scale=3)
]
# Compute dual bounds using y0_dists and y1_dists
dbnd.compute_dual_bounds(
    y0_dists=y0_dists,
    y1_dists=y1_dists,
    suppress_warning=True,
)

100%|████████████████████████████████████████████████████████| 900/900 [00:01<00:00, 733.14it/s]


{'estimates': array([0.29014241, 1.22454801]),
 'ses': array([0.03908947, 0.02725067]),
 'cis': array([0.21352846, 1.27795834])}

This syntax can be useful if in simulations one wants to compute an "oracle dual bound" which has perfect knowledge of the conditional distributions of $Y(0) \mid X$ and $Y(1) \mid X$, as illustrated below.

In [9]:
# Compute oracle dual bounds using the true conditional dists of Y0/Y1
dbnd.compute_dual_bounds(
    y0_dists=data['y0_dists'],
    y1_dists=data['y1_dists'],
    suppress_warning=True,
)

100%|████████████████████████████████████████████████████████| 900/900 [00:01<00:00, 481.40it/s]


{'estimates': array([0.58061888, 0.92968078]),
 'ses': array([0.02343839, 0.01286777]),
 'cis': array([0.53468048, 0.95490115])}

Note that the output of the oracle dual bounds is extremely similar to the output of the initial dual bounds in the third cell.

### Choosing the propensity score model for observational data

Dual bounds can also apply to observational data where the propensity scores must be estimated. In this case, analysts can specify the model used to estimate the propensity scores---the ``W_model``---with one of three methods. First, one can use a string identifier:

In [10]:
dbnd = DualBounds(
    W_model='ridge', # logistic ridge for prop. scores
    Y_model='lasso',
    f=lambda y0, y1, x: y0 < y1, # estimand
    X=data['X'], W=data['W'], y=data['y'], # data
)
dbnd.compute_dual_bounds()

100%|████████████████████████████████████████████████████████| 900/900 [00:01<00:00, 568.16it/s]


{'estimates': array([0.58088302, 0.93676891]),
 'ses': array([0.023641 , 0.0133706]),
 'cis': array([0.53454751, 0.96297481])}

Second, one can directly input an sklearn classifier.

In [11]:
dbnd = DualBounds(
    W_model=ensemble.AdaBoostClassifier(), 
    f=lambda y0, y1, x: y0 < y1, # estimand
    X=data['X'], W=data['W'], y=data['y'], # data
)
dbnd.compute_dual_bounds()

100%|████████████████████████████████████████████████████████| 900/900 [00:01<00:00, 614.49it/s]


{'estimates': array([0.5770192 , 0.92539417]),
 'ses': array([0.02351094, 0.01315318]),
 'cis': array([0.5309386 , 0.95117393])}

Lastly, analysts can also directly estimate the vector propensity scores and input them, although analysts should ensure that they are correctly employing cross-fitting in this case to ensure validity.

## Additional estimands and extensions

Dual bounds can apply beyond the settings described in the previous sections. 

### Variance of the CATE

Dual bounds can also be used to *lower-bound* the variance of the conditional average treatment effect $\theta = \text{Var}(E[Y(1) - Y(0) \mid X])$, as shown below.

In [12]:
vdb = db.varcate.VarCATEDualBounds(
    X=data['X'], 
    W=data['W'], 
    y=data['y'], 
    pis=data['pis'],
    Y_model='elasticnet',
)
vdb.compute_dual_bounds()

{'estimate': 7.271835000465313,
 'se': 0.5761754092436733,
 'lower_ci': 6.324110788810609}

### Variance of the ITE

Dual bounds can also be used to upper and lower bound the variance of the individual treatment effect $\theta = \text{Var}(Y(1) - Y(0))$, as shown below.

In [13]:
vdb = db.varite.VarITEDualBounds(
    X=data['X'], 
    W=data['W'], 
    y=data['y'], 
    pis=data['pis'],
    Y_model='elasticnet',
)
vdb.compute_dual_bounds()

100%|████████████████████████████████████████████████████████| 900/900 [00:02<00:00, 338.36it/s]


{'estimates': array([ 7.18208094, 16.10882366]),
 'ses': array([0.56993296, 0.86895712]),
 'cis': array([ 6.06503287, 17.81194832])}

### Lee Bounds under monotonicity

Lee bounds are a method to bound the average treatment effect in the face of post-treatment nonrandom sample selection, named in honor of  [Lee (2009)](https://www.jstor.org/stable/40247633). Precisely, we assume we observe the following data:
- Pre-treatment covariates $X_i \in \mathcal{X}$
- A binary treatment $W_i \in \{0,1\}$
- A post-treatment selection indicator $S_i \in \{0,1\}$.
- An outcome $Y_i \in \mathbb{R}$.

Note that both $Y_i$ and $S_i$ have potential outcomes $(Y_i(0), Y_i(1))$ and $(S_i(0), S_i(1))$ since both potentially depend on the treatment.

A classic example is a setting where $W_i$ denotes enrollment in a job training program, $S_i$ denotes whether a subject entered the labor market, and the outcome $Y_i$ are log-wages. A natural estimand in these settings is the average treatment effect for subjects who would have entered the labor market no matter their treatment status; e.g., 

$$E[Y(1) - Y(0) \mid S(1) = S(0) = 1]. $$

Dual bounds can be used to bound this partially identified estimand under the **monotonicity** assumption that $S(1) \ge S(0)$ a.s., as shown below.

In [15]:
# create data
lee_data = db.gen_data.gen_lee_bound_data(
    n=900, # Num. datapoints
    p=30, # Num. covariates
    r2=0.95, # population R^2
    tau=3, # average treatment effect
    sample_seed=123,
)
# fit lee bounds
ldb = db.lee.LeeDualBounds(
    # data
    S=lee_data['S'], 
    X=lee_data['X'], 
    W=lee_data['W'],
    pis=lee_data['pis'], 
    y=lee_data['y'],
    # Model specifications
    Y_model='ridge',
    S_model='monotone_logistic',
)
ldb.compute_dual_bounds()

{'estimates': array([2.08484643, 3.25616402]),
 'ses': array([0.2094036 , 0.19805625]),
 'cis': array([1.67442291, 3.64434714])}

It is also possible to bound this estimand without the monotonicity assumption using the generic ``DualBounds`` class, although we caution that without any other assumptions the bounds might be too wide to be useful. Please see [Ji et al. (2023)](https://arxiv.org/pdf/2310.08115.pdf), Section 2.5 for details.